# Decision Tree Regressor 

## Mục tiêu
- Xây dựng mô hình Decision Tree Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 


## Giới thiệu

Mô hình cây quyết định phân chia không gian đặc trưng thành nhiều vùng và dự đoán bằng giá trị trung bình của từng vùng.

### **Ưu điểm:**

- Dễ hiểu, trực quan.

- Không cần chuẩn hóa dữ liệu.

- Học được quan hệ phi tuyến.

### **Nhược điểm:**

- Rất dễ overfitting.

- Không ổn định: dễ bị ảnh hưởng bởi nhiễu/outliers.

## Bước 1 - Import các thư viện cần thiết

In [2]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [3]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [4]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình Decision Tree 

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Decision Tree.

In [5]:
from scipy.stats import randint, uniform

# Định nghĩa không gian siêu tham số để tìm kiếm
param_distributions = {
    "max_depth": randint(2, 40),
    "min_samples_split": randint(2, 50),
    "min_samples_leaf": randint(1, 20),
    "max_features": uniform(0.3, 0.7),   # 0.3 → 1.0
    "min_weight_fraction_leaf": uniform(0.0, 0.1),
    "max_leaf_nodes": randint(10, 200),
    "min_impurity_decrease": uniform(0.0, 0.02)
}

# Tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_distributions=param_distributions,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    n_iter=60,
    random_state=42
)

# Thực hiện tìm kiếm
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END max_depth=30, max_features=0.4284043529063146, max_leaf_nodes=81, min_impurity_decrease=0.011973169683940733, min_samples_leaf=7, min_samples_split=20, min_weight_fraction_leaf=0.00999749158180029; total time=   0.0s
[CV] END max_depth=30, max_features=0.4284043529063146, max_leaf_nodes=81, min_impurity_decrease=0.011973169683940733, min_samples_leaf=7, min_samples_split=20, min_weight_fraction_leaf=0.00999749158180029; total time=   0.0s
[CV] END max_depth=4, max_features=0.9019582847154244, max_leaf_nodes=144, min_impurity_decrease=0.0034104824737458306, min_samples_leaf=7, min_samples_split=19, min_weight_fraction_leaf=0.09488855372533334; total time=   0.0s
[CV] END max_depth=15, max_features=0.8658781436815228, max_leaf_nodes=18, min_impurity_decrease=0.0003193250444042839, min_samples_leaf=2, min_samples_split=21, min_weight_fraction_leaf=0.024102546602601173; total time=   0.0s
[CV] END max_depth=12, max_feat

,estimator,DecisionTreeR...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....x796ff9dd9100>, 'max_features': <scipy.stats....x796ff83fd6d0>, 'max_leaf_nodes': <scipy.stats....x796ff83fd8b0>, 'min_impurity_decrease': <scipy.stats....x796ff83fd940>, ...}"
,n_iter,60
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [6]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_dt = random_search.best_estimator_
best_params = random_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về cây tối ưu
print(f"\nĐộ sâu của cây tối ưu: {best_dt.get_depth()}")
print(f"Số lượng lá: {best_dt.get_n_leaves()}")

SIÊU THAM SỐ TỐI ƯU
  max_depth           : 21
  max_features        : 0.7064806350055182
  max_leaf_nodes      : 121
  min_impurity_decrease: 0.002029430857320642
  min_samples_leaf    : 3
  min_samples_split   : 26
  min_weight_fraction_leaf: 0.0005061583846218687

Độ sâu của cây tối ưu: 14
Số lượng lá: 121


### 4.3. Đánh giá mô hình tối ưu trên tập train

In [7]:
y_pred = best_dt.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Decision Tree:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 score: {train_r2:.3f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Decision Tree:
RMSE loss: 2.230
MAE loss: 1.635
R2 score: 0.936


### 4.4. Lưu mô hình Decision Tree đã tối ưu vào file để sử dụng sau này.

In [8]:
# Tạo thư mục lưu model nếu chưa có
os.makedirs('../models/3_decision_tree', exist_ok=True)

# Lưu mô hình
model_path = '../models/3_decision_tree/decision_tree.pkl'
joblib.dump(best_dt, model_path) 

['../models/3_decision_tree/decision_tree.pkl']

## Kết luận

### Tổng kết:
- Mô hình Decision Tree đã được xây dựng và tối ưu hóa thành công
- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
- Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2